# Analyzing Sea Level Rise in the Cloud with Coiled and Earthaccess

_This notebook was adapted from [this NASA Earthdata Cloud Cookbook example](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/external/Sea_level_rise_notebook.html)_

## Create cluster on AWS with Coiled

In [ ]:
%%time

# Spin up a Coiled cluster in the same region as our data (us-west-2)
import coiled

cluster = coiled.Cluster(n_workers=20, region="us-west-2")
client = cluster.get_client()
client

## Get data files with `earthaccess`

In [ ]:
# Authenticate my machine with `earthaccess`
import earthaccess

earthaccess.login();

In [ ]:
%%time

# Retrieve data files for the dataset I'm interested in
short_name = "SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205"
results = earthaccess.search_data(
    short_name=short_name,
    cloud_hosted=True,
    temporal=("1990", "2019"),
    # bounding_box=...,
)

## Analyze data on my cluster with Xarray

In [ ]:
# %%time

# import xarray as xr

# ds = xr.open_mfdataset(earthaccess.open(results))
# ds

In [ ]:
%%time

fs = earthaccess.get_s3fs_session(daac="PODAAC")

In [ ]:
%%time

import xarray as xr

ds = xr.open_dataset(
    "reference://",
    engine="zarr",
    chunks={},
    backend_kwargs={
        "consolidated": False,
        "storage_options": {
            "fo": f"s3://oss-shared-scratch/jrbourbeau/{short_name}/combined.json",
            "remote_protocol": "s3",
            "remote_options": fs.storage_options,
        }
    },
)
ds

In [ ]:
ds.SLA.where((ds.SLA>=0) & (ds.SLA < 10)).std("Time")[::-1, :].plot.imshow(figsize=(14, 6));